#### Import Packages

In [4]:
import pandas as pd
import numpy as np
import re

#### Loading and cleaning up data and remove unnecessary features

In [5]:
#load cbecs data
data_cbecs = pd.read_csv('cbecs2018_final_public.csv')
#load data definitions including column name definitions
data_def_cbecs = pd.read_excel('2018microdata_codebook.xlsx',skiprows=1)

In [6]:
#cbecs data has 6436 rows and 1249 features
data_cbecs.shape

(6436, 1249)

In [102]:
#create a copy of cbecs data
data_cbecs_mod = data_cbecs.copy(deep=True)

In [103]:
# adding Intensity variable as y variable
data_cbecs_mod['EL Intensity'] = data_cbecs_mod['ELBTU']/data_cbecs_mod['SQFT']

In [104]:
#modifying the column names in data def to remove \n string
data_def_cbecs.rename(columns=lambda x:x.replace('\n',' '),inplace=True)

In [105]:
#find columns with intermediate weight and imputation information. These columns are irrelevant and can be dropped
mask_for_col_drop = data_def_cbecs['Label'].str.contains('Weight|Imputation').fillna(False)
candidate_cols_to_drop = list(data_def_cbecs.loc[mask_for_col_drop]['Variable name'])
#making sure that final weight columns are not dropped
candidate_cols_to_drop.remove('FINALWT')
candidate_cols_to_drop.append('PUBID')
cols_to_drop = list(set(candidate_cols_to_drop) & set(data_cbecs.columns))

In [106]:
#drop columns
data_cbecs_mod.drop(cols_to_drop,axis=1,inplace=True)

In [107]:
# more than half the columns are dropped resulting in 6436 rows and 620 columns
data_cbecs_mod.shape

(6436, 621)

In [108]:
list_cols = list(data_cbecs_mod.columns)
exp_vars = [x for x in list_cols if bool(re.findall('(EL|MF|NG|DH|FK).*(BTU|CNS|EXP)',x))]

In [109]:
data_cbecs_mod.drop(exp_vars,axis=1,inplace=True)

In [110]:
data_cbecs_mod.shape

(6436, 573)

In [111]:
#drop Intensity from potential x variables
x_vars_pot = data_cbecs_mod.columns.drop('EL Intensity')

#### Cleaning up further to select only features with low rates of null values

In [112]:
list_per_missing = []
for x in x_vars_pot:
    missing_per = sum(~data_cbecs_mod[x].notnull())/data_cbecs_mod[x].shape[0] + data_cbecs_mod[data_cbecs_mod[x]=='Missing'].shape[0]/data_cbecs_mod[x].shape[0]
    list_per_missing.append(missing_per)
    
x_missing_per = dict(zip(x_vars_pot,list_per_missing))

In [113]:
x_missing_low = [key for key,value in x_missing_per.items() if value <= 0.10 ]

In [114]:
len(x_missing_low)

206

In [115]:
#mask for getting only those features that have low rates of nulls
mask = data_def_cbecs['Variable name'].isin(x_missing_low)

In [116]:
# for easy reference, create a dictionary with Variable names as key and possible values as value
var_vals = dict(zip(data_def_cbecs[mask]['Variable name'], data_def_cbecs[mask]['Values/Format codes']))
# for easy reference, create a dictionary with Variables names as key and Variable Label (def) as value
var_def = dict(zip(data_def_cbecs[mask]['Variable name'], data_def_cbecs[mask]['Label']))

In [118]:
# remove all columns except one with low nulls and one contating y variable 'EL Intensity'
cols_to_drop = set(data_cbecs_mod.columns) - set(var_vals.keys()) - {'EL Intensity'}

In [119]:
data_cbecs_mod.drop(columns=cols_to_drop,inplace=True)

In [120]:
data_cbecs_mod.shape

(6436, 207)

In [121]:
#ensuring that FINALWT key does not have null values
var_vals['FINALWT']='0-1000000'

#### Assigning Variables into 3 categories : (i)non-binary and non-categoricals (ii)binary, and (iii) categoricals

In [122]:
non_bin_cat_vars = ['SQFT', 'GLSSPC','NFLOOR','FLCEILHT','YRCONC','NOCC','WKHRS','WKHRSC','NWKER',
                    'NWKERC','HEATP','COOLP','LTOHRP','LTNHRP','DAYLTP','HDD65','CDD65','FINALWT','EL Intensity']

#creating a dict of non-binary/non-cat variables with potential values as value. 
#There are 18 variables including weights.
non_bin_cat_vars_dict ={k:v for k,v in var_vals.items() if k in non_bin_cat_vars}
#find binary variables
bin_vars = [k for k,v in var_vals.items() if len(list(re.findall('(\d+)=',v)))== 2 and k not in non_bin_cat_vars]
#dictionary of binary variables
bin_vars_dict = {k:v for k,v in var_vals.items() if k in bin_vars}
#non-binary categorical variables
cat_vars = [k for k,v in var_vals.items() if k not in non_bin_cat_vars and k not in bin_vars]
#non-binary categorical variables dict
cat_vars_dict = {k:v for k,v in var_vals.items() if k in cat_vars}


In [123]:
len(cat_vars_dict)

21

In [124]:
#dict to replace 2 (meaning False) as 0 in binary variables
bin_vars_replace_2_as_0_dict = {k:2 for k in bin_vars}
data_cbecs_mod.replace(bin_vars_replace_2_as_0_dict,0,inplace=True)

In [125]:
# the cleaned up data set has 6436 observations and 206 features
data_cbecs_mod.shape

(6436, 207)

### Finalizing data with dummies (one-hot encoding) and final clean up

In [126]:
data_cbecs_with_dummies = pd.get_dummies(data=data_cbecs_mod,columns=cat_vars)

In [129]:
data_cbecs_with_dummies.shape

(6436, 419)

In [133]:
#drop null y values
mask_for_null_y = data_cbecs_with_dummies['EL Intensity'].isna()
data_cbecs_with_dummies = data_cbecs_with_dummies[~mask_for_null_y]

In [134]:
data_cbecs_final = data_cbecs_with_dummies.copy(deep = True)

In [136]:
data_cbecs_final.shape

(6357, 419)

In [137]:
data_cbecs_final.to_excel('data_cbecs_final.xlsx',index=False)